<a href="https://colab.research.google.com/github/dhyaneesh/Sentinel-1-Man-Made-Change-Detection/blob/main/Change_Detection_and_Edge_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# SAR Change Detection Analysis
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='norse-fragment-435517-g1')

In [24]:
def run_change_detection(roi_geojson, start_date, end_date, significance_level=0.01):
    aoi = ee.Geometry.Polygon(roi_geojson['features'][0]['geometry']['coordinates'])

    # Filter and prepare image collection
    image_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                        .filterBounds(aoi)
                        .filterDate(ee.Date(start_date), ee.Date(end_date))
                        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                        .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd'))))

    # Clip images to area of interest
    clipped_images = image_collection.toList(image_collection.size()).map(lambda img: ee.Image(img).clip(aoi))

    # Select VV polarization
    vv_images = clipped_images.map(lambda img: ee.Image(img).select('VV'))

    # Create RGB composite
    rgb_composite = (ee.Image.rgb(vv_images.get(0), vv_images.get(1), vv_images.get(2))
                     .log10().multiply(10))

    # Perform change detection
    k = len(clipped_images.getInfo())
    p_value = ee.Image.constant(1).subtract(chi2cdf(omnibus(vv_images), k - 1))
    change_map = p_value.lt(significance_level)

    # Create connected components
    object_id = change_map.connectedComponents(connectedness=ee.Kernel.plus(4), maxSize=128)

    return {
        'aoi': aoi,
        'rgb_composite': rgb_composite,
        'change_map': change_map,
        'object_id': object_id
    }
def chi2cdf(chi2, df):
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def omnibus(im_list, m=4):
    im_list = ee.List(im_list)
    k = im_list.length()
    k_log_k = k.multiply(k.log())
    k_log_k = ee.Image.constant(k_log_k)
    sum_logs = ee.ImageCollection(im_list.map(lambda img: ee.Image(img).log())).reduce(ee.Reducer.sum())
    log_sum = ee.ImageCollection(im_list).reduce(ee.Reducer.sum()).log()
    return k_log_k.add(sum_logs).subtract(log_sum.multiply(k)).multiply(-2*m)

In [25]:
roi_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [
                        [79.50457917646156, 18.035453883027586],
                        [79.50457917646156, 17.97513358623202],
                        [79.60597934520041, 17.97513358623202],
                        [79.60597934520041, 18.035453883027586],
                        [79.50457917646156, 18.035453883027586]
                    ]
                ],
                "type": "Polygon"
            }
        }
    ]
}

# Set analysis parameters
start_date = '2022-01-01'
end_date = '2024-09-27'
significance_level = 1e-200  # Extremely low significance level

# Run change detection
result_layers = run_change_detection(roi_geojson, start_date, end_date, significance_level)

# Create a map to display the results
Map = geemap.Map()
Map.centerObject(result_layers['aoi'], 10)

# Add layers to the map
Map.addLayer(result_layers['rgb_composite'], {'min': -20, 'max': 0}, 'RGB Composite')
Map.addLayer(result_layers['change_map'].updateMask(result_layers['change_map']),
             {'min': 0, 'max': 1, 'palette': ['black', 'red']},
             'Change Map')
Map.add_layer(result_layers['object_id'].randomVisualizer(), None, 'Objects')

# Display the map
display(Map)

Map(center=[18.005298608826045, 79.55527926083023], controls=(WidgetControl(options=['position', 'transparent_…